In [ ]:
from sentence_transformers import SentenceTransformer, util
import pickle
import pandas as pd

In [ ]:
embedder = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
embedder.max_seq_length

In [ ]:
data = pd.read_csv("data/cleaned-v2.csv")
descs = data.description.tolist()

In [ ]:
corpus_embeddings = embedder.encode(descs, show_progress_bar=True)

In [ ]:
with open('desc-embeddings.pkl', "wb") as fOut:
    pickle.dump(corpus_embeddings, fOut)

In [ ]:
prompt = "the cat"
prompt_embedding = embedder.encode(prompt, convert_to_tensor=True)

In [ ]:
prompt_embedding.shape

In [ ]:
hits = util.semantic_search(prompt_embedding, corpus_embeddings, top_k=10)
hits = pd.DataFrame(hits[0], columns=['corpus_id', 'score'])

In [ ]:
for idx,score in zip(hits.corpus_id.tolist(), hits.score.tolist()):
    print(data.iloc[idx].romaji, score)

In [ ]:
id = hits.corpus_id.tolist()[0]
row = data.iloc[id]

In [ ]:
row

In [ ]:
row["cover"]

In [ ]:
from PIL import Image
import requests
from io import BytesIO

response = requests.get(row["cover"])
img = Image.open(BytesIO(response.content))

In [ ]:
img

In [ ]:
import streamlit as st
import streamlit.components.v1 as components

components.html(
    """
    <table boarder="1" cellpadding="0" cellspacing="0" width="100%">
        <tr id="header">
            <td width="15%">Rank</td>
            <td width="55%">Title</td>
            <td width="15%">Score</td>
            <td width="15%">Members</td>
        </tr>
    </table>
        <tr class="entry">
            <td class="rank">1</td>
            <td class="title">
                <div class="titlecontents">
                    <div class="titleimage">
                        <img src="https://myanimelist.cdn-dena.com/r/167x242/images/anime/12/86676.webp?s=0b23d3d85a843719c4885a45a3ff9403" />
                    </div>
                    <div class="titleinfo">
                        <a class = "name" href="https://myanimelist.net/anime/34902/Tsurezure_Children">Tsurezure Children</a>
                        <p class="season">Summer 2017</p>
                        <p class="type">TV</p>
                    </div>
                </div>

            </td>
            <td class="score">9.50</td>
            <td class="members">1000000</td>
        </tr>
    <table class="rankingTable" boarder="0" cellpadding="0" cellspacing="0" width="100%">


    </table>

    """,
    height=600,
)
st.show()